DataLab custom function example
===============================

This is part of the DataLab's custom function tutorial which aims at illustrating the extensibility of DataLab (macros, plugins, and control from an external IDE or a Jupyter notebook). 

The only requirement is to install the *DataLab Simple Client* package (using `pip install cdlclient`, for example).

In [2]:
import numpy as np
import scipy.ndimage as spi
from cdlclient import SimpleRemoteProxy

# Define our custom processing function
def weighted_average_denoise(data: np.ndarray) -> np.ndarray:
    """Apply a custom denoising filter to an image.

    This filter averages the pixels in a 5x5 neighborhood, but gives less weight
    to pixels that significantly differ from the central pixel.
    """

    def filter_func(values: np.ndarray) -> float:
        """Filter function"""
        central_pixel = values[len(values) // 2]
        differences = np.abs(values - central_pixel)
        weights = np.exp(-differences / np.mean(differences))
        return np.average(values, weights=weights)

    return spi.generic_filter(data, filter_func, size=5)

Connecting to DataLab current session:

In [3]:
proxy = SimpleRemoteProxy()
proxy.connect()

Connecting to DataLab XML-RPC server...OK (port: 61330)


Switch to the "Image Panel" and get the current image

In [5]:
proxy.set_current_panel("image")
image = proxy.get_object()
if image is None:
    # We raise an explicit error if there is no image to process
    raise RuntimeError("No image to process!")

Get a copy of the image data, apply the function to it, and add new image to the panel

In [6]:
data = np.array(image.data, copy=True)
data = weighted_average_denoise(data)
proxy.add_image("Filtered using a Jupyter notebook", data)

True